## Interactive tactics board with pitch control

Click and drag the dots to change player positions. Click and drag the arrow heads to change player velocities. Interactive visualisation based on [bqplot](https://github.com/bqplot/bqplot). 
There are three available pitch control models:
- Spearman (Beyond Expected Goals, Sloan Sports Conference, 2018). The code for this is slightly modified from [Laurie Shaw's implementation](https://github.com/Friends-of-Tracking-Data-FoTD/LaurieOnTracking/blob/master/Metrica_PitchControl.py). This model will take longer to update after moving the players.
- Fernandez (Wide Open Spaces: A statistical technique for measuring space creation in professional soccer, 2018).
- 'Modified Fernandez' -- this is almost identical to the Fernandez model, but the control of a player over a location is replaced by their proportion of the total 'influence' (as defined in the Fernandez paper) at that location.


All of the code for computing pitch control and the interactive visualisations is available [here](https://github.com/anenglishgoat/InteractivePitchControl). I converted the .ipynb in that repo to a standalone web app using [Voilà](https://github.com/voila-dashboards/voila).

In [9]:
import numpy as np
import ipywidgets
from bqplot import *
import bqplot.pyplot as plt
import matplotlib.pyplot as plt2
from scipy.special import expit

def get_contours(Z,x,y,n_contour = 10):
    levels = np.linspace(0,np.max(Z),n_contour)
    cont = plt2.contourf(xx,yy,Z,levels = np.linspace(0,np.max(Z),n_contour+1),nchunk=10)
    plt2.close()
    contours = [cs.get_paths() for cs in cont.collections]
    levels = [np.repeat(levels[i],len(contours[i])) for i in range(n_contour)]
    contours = [item for sublist in contours for item in sublist]
    contours = [c.vertices for c in contours]
    contours_x = [list(c[:,0])  for c in contours]
    contours_y = [list(c[:,1]) for c in contours]
    return contours_x,contours_y, levels

def get_pitch_control_proportion(Z):
    pc = np.sum(Z) / np.prod(Z.shape) * 100
    return '%s' % float('%.3g' % pc) + '%'

def get_pitch_control(home_pos,
                      away_pos,
                      home_v,
                      away_v,
                      ball_pos,
                      target_position,
                      x_grid,
                      y_grid,
                      xT_values,
                      transition,
                      value):
    reaction_time = 0.7
    max_player_speed = 5.
    average_ball_speed = 15.
    time_to_control = 3. * np.log(10.) * (np.sqrt(3) * 0.45 + 1/4.3)    

    max_int_time = np.array([0.,10.])
    
    ball_travel_time = np.linalg.norm(xxyy - ball_pos,axis = 2)/average_ball_speed

    # first get arrival time of 'nearest' attacking player (nearest also dependent on current velocity)

    r_reaction_home = np.expand_dims(home_pos + reaction_time * home_v, 1)
    r_reaction_away = np.expand_dims(away_pos + reaction_time * away_v, 1)

    tti_home = reaction_time + np.sqrt(
        np.sum((np.expand_dims(target_position, 0) - np.expand_dims(r_reaction_home, 1)) ** 2.,
               axis=3)) / max_player_speed
    tti_away = reaction_time + np.sqrt(
        np.sum((np.expand_dims(target_position, 0) - np.expand_dims(r_reaction_away, 1)) ** 2.,
               axis=3)) / max_player_speed
    tti = np.concatenate((tti_home, tti_away))

    y = np.zeros_like(tti)

    for tt in np.arange(0, 10, 0.02):
        sumy = np.expand_dims(np.sum(y, axis=0), 0)
        if np.min(sumy) > 0.99:
            break
        y += 0.02 * 4.3 * (1. - sumy) * 1. / (1. + np.exp(-np.pi / np.sqrt(3.) / 0.45 * (tt + ball_travel_time - tti)))
    
    out = np.sum(y[:11], axis=0)
    
    if transition == 'Yes':
        gaussian_x = np.exp(-0.5*(x_grid - ball_pos[0][0])**2/(23.9/105*100)**2)
        gaussian_y = np.exp(-0.5*(y_grid - ball_pos[0][1])**2/(23.92/68*100)**2)
        gaussian = np.outer(gaussian_x,gaussian_y)
        transition_prob = gaussian * out
        transition_prob /= np.sum(4*transition_prob)
        out = out * transition_prob
    if value == 'Yes':
        out = out * xT_values
        
    return out

def get_pitch_control_fernandez(home_pos,
                                away_pos,
                                home_v,
                                away_v,
                                ball_pos,
                                target_position,
                                x_grid,
                                y_grid,
                                xT_values,
                                transition,
                                value):
    s_home = np.linalg.norm(home_v,axis=1,keepdims=True)
    s_away = np.linalg.norm(away_v,axis=1,keepdims=True)
    theta_home = np.arccos(home_v[:,0,None] / s_home)
    theta_away = np.arccos(away_v[:,0,None] / s_away)
    mu_home = home_pos + 0.5*home_v
    mu_away = away_pos + 0.5*away_v
    Srat_home = np.minimum((s_home / 13.0)**2,1.)
    Srat_away = np.minimum((s_away / 13.0)**2,1.)
    Ri_home = np.minimum(4 + np.linalg.norm(ball_pos - home_pos,axis=1,keepdims=True)**3 / 972,10.)
    Ri_away = np.minimum(4 + np.linalg.norm(ball_pos - away_pos,axis=1,keepdims=True)**3 / 972,10.)
    RSinv_home = np.empty((11,2,2))
    RSinv_away = np.empty((11,2,2))
    S1_home = 2 / ((1+Srat_home) * Ri_home)
    S2_home = 2 / ((1-Srat_home) * Ri_home)
    S1_away = 2 / ((1+Srat_away) * Ri_away)
    S2_away = 2 / ((1-Srat_away) * Ri_away)
    RSinv_home[:,0,0] = (S1_home * np.cos(theta_home))[:,0]
    RSinv_home[:,1,0] = (S1_home * np.sin(theta_home))[:,0]
    RSinv_home[:,0,1] = - (S2_home * np.sin(theta_home))[:,0]
    RSinv_home[:,1,1] = (S2_home * np.cos(theta_home))[:,0]
    RSinv_away[:,0,0] = (S1_away * np.cos(theta_away))[:,0]
    RSinv_away[:,1,0] = (S1_away * np.sin(theta_away))[:,0]
    RSinv_away[:,0,1] = - (S2_away * np.sin(theta_away))[:,0]
    RSinv_away[:,1,1] = (S2_away * np.cos(theta_away))[:,0]
    denominators_h = np.exp(-0.5 * np.sum((np.matmul(home_pos[:,None,:] - mu_home[:,None,:],RSinv_home))**2,-1))
    denominators_a = np.exp(-0.5 * np.sum((np.matmul(away_pos[:,None,:] - mu_away[:,None,:],RSinv_away))**2,-1))
    x_min_mu_h = mu_home[:,None,None,:] - targets[None,:,:,:]
    x_min_mu_a = mu_away[:,None,None,:] - targets[None,:,:,:]
    mm_h = np.matmul(RSinv_home[:,None,None,:],x_min_mu_h[:,:,:,:,None])[...,0]
    mm_a = np.matmul(RSinv_away[:,None,None,:],x_min_mu_a[:,:,:,:,None])[...,0]
    infl_h = np.exp(-0.5 * np.sum(mm_h**2,-1)) / denominators_h[...,None]
    infl_a = np.exp(-0.5 * np.sum(mm_a**2,-1)) / denominators_a[...,None]
    
    out = expit(np.sum(infl_h,0)-np.sum(infl_a,0))
    
    if transition == 'Yes':
        gaussian_x = np.exp(-0.5*(x_grid - ball_pos[0][0])**2/(23.9/105*100)**2)
        gaussian_y = np.exp(-0.5*(y_grid - ball_pos[0][1])**2/(23.92/68*100)**2)
        gaussian = np.outer(gaussian_x,gaussian_y)
        transition_prob = gaussian * out
        transition_prob /= np.sum(4*transition_prob)
        out = out * transition_prob
    if value == 'Yes':
        out = out * xT_values
        
    return out

def get_pitch_control_fernandez_modified(home_pos,
                                         away_pos,
                                         home_v,
                                         away_v,
                                         ball_pos,
                                         target_position,
                                         x_grid,
                                         y_grid,
                                         xT_values,
                                         transition,
                                         value):
    s_home = np.linalg.norm(home_v,axis=1,keepdims=True)
    s_away = np.linalg.norm(away_v,axis=1,keepdims=True)
    theta_home = np.arccos(home_v[:,0,None] / s_home)
    theta_away = np.arccos(away_v[:,0,None] / s_away)
    mu_home = home_pos + 0.5*home_v
    mu_away = away_pos + 0.5*away_v
    Srat_home = np.minimum((s_home / 13.0)**2,1.)
    Srat_away = np.minimum((s_away / 13.0)**2,1.)
    Ri_home = np.minimum(4 + np.linalg.norm(ball_pos - home_pos,axis=1,keepdims=True)**3 / 972,10.)
    Ri_away = np.minimum(4 + np.linalg.norm(ball_pos - away_pos,axis=1,keepdims=True)**3 / 972,10.)
    RSinv_home = np.empty((11,2,2))
    RSinv_away = np.empty((11,2,2))
    S1_home = 2 / ((1+Srat_home) * Ri_home)
    S2_home = 2 / ((1-Srat_home) * Ri_home)
    S1_away = 2 / ((1+Srat_away) * Ri_away)
    S2_away = 2 / ((1-Srat_away) * Ri_away)
    RSinv_home[:,0,0] = (S1_home * np.cos(theta_home))[:,0]
    RSinv_home[:,1,0] = (S1_home * np.sin(theta_home))[:,0]
    RSinv_home[:,0,1] = - (S2_home * np.sin(theta_home))[:,0]
    RSinv_home[:,1,1] = (S2_home * np.cos(theta_home))[:,0]
    RSinv_away[:,0,0] = (S1_away * np.cos(theta_away))[:,0]
    RSinv_away[:,1,0] = (S1_away * np.sin(theta_away))[:,0]
    RSinv_away[:,0,1] = - (S2_away * np.sin(theta_away))[:,0]
    RSinv_away[:,1,1] = (S2_away * np.cos(theta_away))[:,0]
    denominators_h = np.exp(-0.5 * np.sum((np.matmul(home_pos[:,None,:] - mu_home[:,None,:],RSinv_home))**2,-1))
    denominators_a = np.exp(-0.5 * np.sum((np.matmul(away_pos[:,None,:] - mu_away[:,None,:],RSinv_away))**2,-1))
    x_min_mu_h = mu_home[:,None,None,:] - targets[None,:,:,:]
    x_min_mu_a = mu_away[:,None,None,:] - targets[None,:,:,:]
    mm_h = np.matmul(RSinv_home[:,None,None,:],x_min_mu_h[:,:,:,:,None])[...,0]
    mm_a = np.matmul(RSinv_away[:,None,None,:],x_min_mu_a[:,:,:,:,None])[...,0]
    infl_h = np.exp(-0.5 * np.sum(mm_h**2,-1)) / denominators_h[...,None]
    infl_a = np.exp(-0.5 * np.sum(mm_a**2,-1)) / denominators_a[...,None]
    infl_all = np.concatenate([infl_h,infl_a])
    infl_all /= np.sum(infl_all,0)
    
    out = np.sum(infl_all[:11],0)
    
    if transition == 'Yes':
        gaussian_x = np.exp(-0.5*(x_grid - ball_pos[0][0])**2/(23.9/105*100)**2)
        gaussian_y = np.exp(-0.5*(y_grid - ball_pos[0][1])**2/(23.92/68*100)**2)
        gaussian = np.outer(gaussian_x,gaussian_y)
        transition_prob = gaussian * out
        transition_prob /= np.sum(4*transition_prob)
        out = out * transition_prob
    if value == 'Yes':
        out = out * xT_values
        
    return out

In [2]:
xT_values = np.array([[0.001290462240925205,
  0.0014789730471282936,
  0.0016395981440987467,
  0.0017839046364342376,
  0.001923459628732441,
  0.002069830225591032,
  0.002234583531607684,
  0.0024288047058457912,
  0.002650545301607838,
  0.0028836387763905005,
  0.0031112281414785205,
  0.0033225957939458277,
  0.0035207219114243033,
  0.0037104400710293575,
  0.003898325018714041,
  0.004099462541837841,
  0.0043315349521347414,
  0.004610627841017703,
  0.004934257549678058,
  0.005287960257873844,
  0.005657167365968339,
  0.006030999121616032,
  0.006403367715576372,
  0.006768517108424999,
  0.007127717577889099,
  0.007501189297000735,
  0.007912315073248423,
  0.008382103547929868,
  0.008917337816718808,
  0.00951952637120278,
  0.010190060188973135,
  0.010928701246664896,
  0.01173395751275115,
  0.01260403492508883,
  0.013520953031245323,
  0.014441642473839193,
  0.015320880082985898,
  0.016131336467923883,
  0.01690433552708446,
  0.017683200310881678,
  0.01849390954386518,
  0.019234256542242728,
  0.019744581542796107,
  0.019874798234437507,
  0.019669382928402007,
  0.019351165813251293,
  0.019149572094787553,
  0.01929402697881297,
  0.020013955671129742,
  0.021538783377540043],
 [0.0012962071386798237,
  0.0015112913258124583,
  0.0017012873903125257,
  0.0018737080074205407,
  0.0020360658523770167,
  0.002195873600422468,
  0.002360643926797407,
  0.00253780258818328,
  0.002732424739063504,
  0.002947021309725519,
  0.0031839407100356146,
  0.0034385828167130244,
  0.003690844411295128,
  0.0039185246049352855,
  0.004107825597810023,
  0.004286024893691939,
  0.004492931133482927,
  0.004765110736324114,
  0.005101403693566324,
  0.0054763103657279365,
  0.0058641529503640245,
  0.006248097612308382,
  0.006622799142698969,
  0.006983692782203222,
  0.007333928982872865,
  0.007697466031765364,
  0.008101734930425231,
  0.008570923571183664,
  0.009109787761854752,
  0.009715878205077258,
  0.010386762697587851,
  0.011120441515516874,
  0.011915247858488319,
  0.012769462304900827,
  0.013677694322728442,
  0.014628863158787014,
  0.015611395677661238,
  0.016601848501549107,
  0.017537869268989658,
  0.01834914571013748,
  0.018970807539751458,
  0.019378204140056393,
  0.019564711467278756,
  0.01952835894254407,
  0.019357465015354892,
  0.01922311530857333,
  0.019299455938858626,
  0.01976063302287006,
  0.020780792677266884,
  0.022534081018708338],
 [0.0013668369125675037,
  0.0015885164887259723,
  0.001792480335816615,
  0.0019821093336048713,
  0.0021607843618561837,
  0.002331886300335993,
  0.002498796028809739,
  0.0026650949108658626,
  0.0028397861408278983,
  0.003037787478048314,
  0.0032742462720493754,
  0.0035495389181350104,
  0.003831085861867296,
  0.004081848392817863,
  0.004276574361501793,
  0.004447625737214921,
  0.0046449412412139365,
  0.00491455984453823,
  0.005257135987996717,
  0.005644043770314126,
  0.0060464023982532795,
  0.006444393375199536,
  0.006829970448382318,
  0.0071958893237780725,
  0.0075441795318507004,
  0.007901881999407723,
  0.008300211919267506,
  0.008766586220741418,
  0.009305663371235407,
  0.009913663365612089,
  0.010586983551383603,
  0.011324881945849933,
  0.012128818715578407,
  0.013000257687270858,
  0.013937483480109553,
  0.014933852941621398,
  0.015982293941847445,
  0.01705493410131386,
  0.018055720750531537,
  0.01887465305660253,
  0.019416049538053405,
  0.019690057670032346,
  0.019754257779281147,
  0.019668514453166818,
  0.019533415735232908,
  0.019486880509965075,
  0.01966820804977945,
  0.0202166976270922,
  0.021271648514319483,
  0.02297235998387741],
 [0.0014860904548324397,
  0.0017005312065181673,
  0.0019070886580907778,
  0.00210528146108265,
  0.0022946282670261663,
  0.0024746477274537066,
  0.00264485849389765,
  0.0028051693713171144,
  0.002966040369296305,
  0.0031494415922449152,
  0.003377839631836985,
  0.0036554904510646786,
  0.003946019695153531,
  0.004207555461047759,
  0.004410728330337797,
  0.004587282524030921,
  0.004787606638926069,
  0.0050583200034191285,
  0.005402171233743736,
  0.0057936052608675735,
  0.006206736891026272,
  0.006621185760909068,
  0.007023722476669044,
  0.007401614271775624,
  0.007753547473574466,
  0.008109005572244057,
  0.008502611936931546,
  0.00896490381592881,
  0.009501935209945113,
  0.010110682124289344,
  0.010788465741005995,
  0.011537994741440277,
  0.012366125114567356,
  0.013279626442079948,
  0.014273682879453964,
  0.015325521171529228,
  0.016410819991994525,
  0.01749155940819748,
  0.01848481738414906,
  0.01929848591714466,
  0.01985263155415382,
  0.020157298373331564,
  0.02026285864862854,
  0.020221679666579357,
  0.020121797897111705,
  0.02008394744283982,
  0.020230071471493388,
  0.02068211315080214,
  0.021562015648495792,
  0.022991722132304042],
 [0.0016377066577188287,
  0.0018372181498383757,
  0.002039024034614782,
  0.0022393972359493015,
  0.002434610677743193,
  0.002620937283897712,
  0.002794649978314114,
  0.0029525136669606125,
  0.0031045982868640083,
  0.003275487963201361,
  0.003490415593715635,
  0.0037564637683549346,
  0.004040219343166577,
  0.004302789835995654,
  0.004516444524866492,
  0.004708012705764075,
  0.0049209686902176334,
  0.00519573605072589,
  0.005537226231581896,
  0.005927439626623444,
  0.0063479776100731815,
  0.006779774120056584,
  0.007202896071601788,
  0.007597375164821957,
  0.00795711105679604,
  0.008313405122590633,
  0.008703800880573676,
  0.009161688676071537,
  0.009695573843068244,
  0.010304734752591133,
  0.010988952257100653,
  0.011755752106064127,
  0.01261862208600115,
  0.013590773939209183,
  0.014659654896827013,
  0.015772777197697443,
  0.016874218944552582,
  0.01791269056317934,
  0.018852086582281227,
  0.01965940785863173,
  0.020303549603435507,
  0.0207674074911151,
  0.021040152245146274,
  0.02111426948305532,
  0.021047174310066995,
  0.020955802132610907,
  0.02095928924794386,
  0.021176771953322668,
  0.021727386546004146,
  0.02273026932324508],
 [0.0018054244134708642,
  0.0019884599893359293,
  0.0021821981428683935,
  0.0023806295043002483,
  0.002577744703863489,
  0.002767534371790109,
  0.0029439891383120993,
  0.0031016154952199313,
  0.003248870755926294,
  0.003409430901803684,
  0.003607668962002508,
  0.003852485222858674,
  0.004118258237919157,
  0.0043746955440322165,
  0.004599879965636318,
  0.004812833734038511,
  0.005045068758686926,
  0.005326152824217593,
  0.0056630177822857035,
  0.006047991656816892,
  0.006472945736784183,
  0.006921457803261664,
  0.0073663320772231755,
  0.007779679541543147,
  0.008149948530267282,
  0.008509649022763703,
  0.00889864464735019,
  0.009352753120495284,
  0.009883549835689157,
  0.010493621521999532,
  0.01118618609031317,
  0.011974126243497685,
  0.012877764660425735,
  0.013916905548539598,
  0.01506876190829401,
  0.01624453036931293,
  0.017349735915971637,
  0.018319293707238792,
  0.019184455757367023,
  0.019996182447931596,
  0.02079179970128127,
  0.021507866264543932,
  0.02203577673865962,
  0.022272698802868324,
  0.022234107783174424,
  0.022043930604691633,
  0.021830104423074242,
  0.021720566393976525,
  0.0218432536730528,
  0.02232610341595732],
 [0.0019729826143327425,
  0.0021441393956601594,
  0.0023305226603313775,
  0.002525151112230913,
  0.0027210434552432813,
  0.0029112183932529984,
  0.00308869463014458,
  0.003246962553518647,
  0.0033922686388784427,
  0.0035447747189379197,
  0.0037252945410147916,
  0.003943581167428798,
  0.00418470981142401,
  0.004430416611528128,
  0.0046671916731957286,
  0.004904763060478366,
  0.00515994820793225,
  0.005448915161653314,
  0.005780262686629663,
  0.006157706140683082,
  0.006584462452549446,
  0.0070475361611438995,
  0.0075128713375758405,
  0.00794503494056527,
  0.008327138142740064,
  0.008692305645079522,
  0.009082009134417403,
  0.00953390946852573,
  0.010062833752892209,
  0.010675142703996636,
  0.01137791023128916,
  0.01218908935751717,
  0.013135007868387071,
  0.014241226639952254,
  0.01547436628992028,
  0.01670969003556261,
  0.017814616022701745,
  0.01871233498135519,
  0.019508852321845422,
  0.02034757325191209,
  0.021340377863073898,
  0.022366155934779062,
  0.02319937029899387,
  0.023623382526291994,
  0.023607161125509674,
  0.023289818884495318,
  0.022816760040827926,
  0.0223333888320865,
  0.021985109495850048,
  0.021917326269697578],
 [0.002124120152548659,
  0.002294139039460398,
  0.002477909264483502,
  0.0026691349058367187,
  0.002861520041738796,
  0.003048768750408483,
  0.003224585110064526,
  0.0033830425392803354,
  0.00352820279811574,
  0.0036750237254901015,
  0.0038389871350696728,
  0.004029777954918205,
  0.0042441474956938655,
  0.004477097064854061,
  0.00472453666809317,
  0.004986818136707772,
  0.005265648401551909,
  0.005563367900792132,
  0.005889677745388285,
  0.006259027867457176,
  0.006685348938759149,
  0.007159308544322882,
  0.007641354696702417,
  0.00808994890051441,
  0.008483758142966254,
  0.008855943361854347,
  0.009248760238931621,
  0.009700970039488579,
  0.010230396159761755,
  0.010847098570064537,
  0.011561867670674228,
  0.012396613651898787,
  0.01338180674043112,
  0.014546942583328205,
  0.015849830417771143,
  0.017137165545633378,
  0.01824610438119296,
  0.01909278052650788,
  0.019852203688155406,
  0.02075234383744107,
  0.0219722801041962,
  0.023329757742981523,
  0.024480571095974325,
  0.02509273555359995,
  0.02509089714614842,
  0.02463495299743527,
  0.023893499145148305,
  0.02303513162697534,
  0.022228446480604187,
  0.021642039743722665],
 [0.002242575920362809,
  0.0024283415913859763,
  0.0026182696328045318,
  0.0028087537312130872,
  0.0029961875732062602,
  0.0031769648453786633,
  0.0033474792343249093,
  0.0035043431499285718,
  0.003650084096033469,
  0.0037936822323462642,
  0.003944441548484339,
  0.004111101938179796,
  0.004301144722741456,
  0.004521880930380689,
  0.004778071970877092,
  0.005062016414350864,
  0.005362210703144206,
  0.005668855879393132,
  0.005991979759336077,
  0.006354401626374334,
  0.006778426376803469,
  0.0072580743034181975,
  0.007750622998645541,
  0.008210928960016635,
  0.00861488677969772,
  0.008995130545404432,
  0.009393763858049159,
  0.009849747152709509,
  0.010383207621382163,
  0.011007289391685321,
  0.011735801399113992,
  0.012592671330418753,
  0.013609616307103846,
  0.014817258748548506,
  0.016168516667911925,
  0.01749586624871215,
  0.01862144610789534,
  0.01946159648367623,
  0.02024143726873596,
  0.021249257771386396,
  0.02271050244003098,
  0.024386152930312318,
  0.02582901729942627,
  0.026607172785065837,
  0.02660987865416634,
  0.026020818968924806,
  0.025034564779978775,
  0.023845687137965827,
  0.022648757093523507,
  0.021638345697289403],
 [0.0023120955708866324,
  0.002536632910753262,
  0.00274551646600038,
  0.0029401804046311794,
  0.0031220588946488494,
  0.003292586104056583,
  0.003453196200857569,
  0.003605353081141276,
  0.003751324599388721,
  0.003894255631961478,
  0.004037353143264371,
  0.004187579166962119,
  0.004360273792190783,
  0.004571910713900734,
  0.004833953418802711,
  0.005133374896844246,
  0.005449676646676746,
  0.0057647241427602235,
  0.00608788530415735,
  0.006446271562577675,
  0.006866515378890721,
  0.007345132904991084,
  0.00783951814220784,
  0.008304484517034727,
  0.008715604554038883,
  0.00910443782286798,
  0.00951188782647585,
  0.009976054506175468,
  0.010518239402988558,
  0.011153515480618242,
  0.01189745396271458,
  0.012773234270317927,
  0.013809892629885388,
  0.015035384750907975,
  0.01640379588246638,
  0.017754712060005205,
  0.018917893542458087,
  0.01981974630556132,
  0.020703467380641292,
  0.02187706216825862,
  0.023578034599380022,
  0.02552283868125453,
  0.027194392303101257,
  0.028093185614471845,
  0.028088772525852252,
  0.02738902451334039,
  0.02621432307082253,
  0.02478504969218493,
  0.023321585871313874,
  0.02204431310209564],
 [0.0023280804923604638,
  0.002614396118845796,
  0.0028553265066549195,
  0.003059536325329375,
  0.003235690244410708,
  0.0033924529334404614,
  0.003538489061960176,
  0.0036822820190846365,
  0.003827412694690642,
  0.003972113941808377,
  0.00411437873621485,
  0.004258713139109607,
  0.0044241547654887085,
  0.004631707444815392,
  0.004896296851591887,
  0.005203137912327676,
  0.005528381482795929,
  0.005850675543494231,
  0.006177722900042579,
  0.006535971406527042,
  0.006951455287708251,
  0.007421983699002151,
  0.007908700432431552,
  0.0083703304659975,
  0.008784875022880974,
  0.009182323134406473,
  0.00960134031431199,
  0.010078082609884685,
  0.010633669829741258,
  0.011283646586437166,
  0.012043801521526263,
  0.012933711401888415,
  0.013975869100780674,
  0.015191849299023749,
  0.016543634388296636,
  0.017900838712108583,
  0.019125151819891755,
  0.020163558853119587,
  0.021242631897559455,
  0.022646140278607573,
  0.024587028245373597,
  0.02675479846753672,
  0.028604345549576835,
  0.02960914024440332,
  0.029631657512940376,
  0.028890969117529943,
  0.027619332752261066,
  0.02604900611122279,
  0.02441224688850416,
  0.02294131277819424],
 [0.0023215009955960563,
  0.002673789914214715,
  0.002948759690113007,
  0.0031627859870881577,
  0.003332244468937393,
  0.0034735107994579404,
  0.0036029606424470223,
  0.003736591466026223,
  0.0038801729159052105,
  0.004028317314056468,
  0.004175109142806809,
  0.004322412720466979,
  0.004489450206953383,
  0.0046977923919134254,
  0.004962992803268693,
  0.005271191874360451,
  0.00559955677505254,
  0.005927504827577101,
  0.006260636867673491,
  0.006621446320440577,
  0.007032090980195254,
  0.007490736004416704,
  0.007964379266717711,
  0.008417963671697979,
  0.008833511369852248,
  0.009239107038403884,
  0.009670522939749657,
  0.010161275110721273,
  0.010731360719994577,
  0.011395764356826626,
  0.012169481513418387,
  0.013067793752929695,
  0.014106202856510971,
  0.01529951622330457,
  0.016620538445680465,
  0.017976969765511372,
  0.019270927326016565,
  0.020475219955307194,
  0.02179437387383441,
  0.023480319501302367,
  0.02571656133841702,
  0.028180893578288055,
  0.030324449557343484,
  0.031615835226119174,
  0.031890109975470274,
  0.03131825790347617,
  0.030083684644932256,
  0.028369795834633945,
  0.026359997107376645,
  0.02423769409795578],
 [0.0023300882586492996,
  0.0027301616624454757,
  0.003027899177866807,
  0.0032458640594230535,
  0.003406619561623977,
  0.003532728938979338,
  0.0036467554459988934,
  0.003770741178577148,
  0.003912634158803544,
  0.004065007469910966,
  0.004219692866910061,
  0.004376283672526214,
  0.00455169029216912,
  0.004765166246030276,
  0.005030748514624399,
  0.005336975008974684,
  0.005664604456300606,
  0.005996214285826567,
  0.006335546437538165,
  0.00669999737449275,
  0.007106698151434205,
  0.0075536150818470175,
  0.008013818802467118,
  0.008458740335626406,
  0.008874579131125491,
  0.009287364915183447,
  0.009729774870513656,
  0.010232454316828801,
  0.010813874042871485,
  0.011487990716625795,
  0.012268686836784727,
  0.013168845815888118,
  0.014200581967976654,
  0.01537549476791923,
  0.01667548037332857,
  0.018036394800070264,
  0.01939014964581827,
  0.020734227162806113,
  0.022281040979026934,
  0.02428697487792631,
  0.02693942525789966,
  0.029915928624705827,
  0.03266550077112486,
  0.034650562619464816,
  0.035619773486616886,
  0.03558418495631225,
  0.03456455112926417,
  0.03258162610618592,
  0.029656163987790772,
  0.025808918874792013],
 [0.0023915734595760852,
  0.002798858729123536,
  0.0030948281314084835,
  0.0033047052118495932,
  0.0034537135158655328,
  0.003567076588874971,
  0.0036700179762965725,
  0.0037871909133485263,
  0.003927825319156761,
  0.004084326130577085,
  0.004248278412394413,
  0.004417931756779295,
  0.004606405196720233,
  0.004828829698001387,
  0.005096271226450275,
  0.0053999255422024954,
  0.0057249264593941555,
  0.006057806209060365,
  0.006401370840124678,
  0.006768925638858039,
  0.0071735524965075795,
  0.007612846191905365,
  0.008064283197080581,
  0.008504016659273039,
  0.00892114384287351,
  0.009339672145068393,
  0.0097894352743288,
  0.010298442536350826,
  0.010883771767494962,
  0.011558447590673862,
  0.012335610390019083,
  0.013230232083210043,
  0.014258694506078093,
  0.015436894177036524,
  0.01674943248995165,
  0.01813240339564195,
  0.019517748364282585,
  0.020920078026298347,
  0.02262498088269784,
  0.02497348145006271,
  0.028228411383210857,
  0.03207470821798734,
  0.03593829563564462,
  0.03925061448428563,
  0.04157629161955515,
  0.04260204436117138,
  0.04201910458568487,
  0.03951870416964621,
  0.034792074989605945,
  0.027530448922114707],
 [0.002540266615677001,
  0.0028939155115628324,
  0.0031516156215775394,
  0.0033358919357240227,
  0.0034692694440051817,
  0.00357427313642392,
  0.003673428002983137,
  0.003788769448190353,
  0.003929092657733378,
  0.004088749331504774,
  0.004261382551894141,
  0.004445340175824076,
  0.004649467033585623,
  0.004884028264762805,
  0.005156351196336588,
  0.005459402776117094,
  0.005781768928163833,
  0.006113208349745634,
  0.006457125672779994,
  0.0068257293579900005,
  0.0072309998118940495,
  0.007670347686390089,
  0.00812225700679512,
  0.008563966845563972,
  0.008984885492169244,
  0.009407241472269222,
  0.009858740830429792,
  0.010365454689510825,
  0.010943658037033946,
  0.011605993610704158,
  0.01236579736416559,
  0.013246866894312048,
  0.014281053191727067,
  0.015499510504946926,
  0.016879158884524483,
  0.018312855374138244,
  0.0196862582724961,
  0.02101793987816106,
  0.022762145871919395,
  0.0254622509706283,
  0.029560884770793328,
  0.034756107463821,
  0.04041226102995271,
  0.04588669071069468,
  0.05042957133495545,
  0.05319283553901039,
  0.05332478324330109,
  0.049973714368269265,
  0.04228792883435653,
  0.02941572656200461],
 [0.0027659926802040924,
  0.003012293991651651,
  0.00320014749964825,
  0.003344430295025316,
  0.003460019468614283,
  0.0035617921112465777,
  0.003664625313753628,
  0.0037831039129217654,
  0.00392390910553049,
  0.004085100168002562,
  0.004264310625651295,
  0.004461399967046104,
  0.004681194098775924,
  0.004929190916337179,
  0.005208858142292106,
  0.005513739771226859,
  0.00583435229494089,
  0.00616237924447874,
  0.006503079582969438,
  0.006870470616822318,
  0.007278297416600003,
  0.007724047207740561,
  0.008184087699828908,
  0.008633395670031132,
  0.009059467888285737,
  0.00948356739883079,
  0.009932592653427613,
  0.010431807882760697,
  0.010996685380211138,
  0.01163906674590739,
  0.012372376697267208,
  0.013233817117727652,
  0.01427889360021092,
  0.015562064508980929,
  0.017046704390563307,
  0.018555007873989303,
  0.019897074858949922,
  0.021086703742784384,
  0.022805387925020574,
  0.025871216278682618,
  0.030973681613118764,
  0.03785187145457279,
  0.04581889886836197,
  0.0541635988360169,
  0.06173469366995971,
  0.06697745819584018,
  0.06832224890296003,
  0.06419942228062125,
  0.05303933481812557,
  0.03327234300477495],
 [0.003027466984264247,
  0.003139016944167826,
  0.003242181486441085,
  0.0033412171884881227,
  0.0034403806277130463,
  0.0035439283815199606,
  0.003656117027312966,
  0.0037811770561492863,
  0.0039226334869180545,
  0.0040832417532486965,
  0.004265716750223783,
  0.004472434350948155,
  0.004705014025717887,
  0.004964972898571617,
  0.005252416678524094,
  0.005560551909241805,
  0.005880480356593212,
  0.006204599630685429,
  0.006540377510382353,
  0.006905004464068396,
  0.007315340082269489,
  0.007769081586178508,
  0.00823903360145598,
  0.008696359647210408,
  0.009125955634541326,
  0.009549753575382104,
  0.009995866605453126,
  0.010490296065343059,
  0.011046389826767691,
  0.011672804058193917,
  0.012380774122633574,
  0.013220242836859391,
  0.014270950092197594,
  0.015611335724222783,
  0.017195847169437475,
  0.018786743645782377,
  0.02012981704603756,
  0.02123681763904014,
  0.022991269813386405,
  0.02645504345715958,
  0.032545227894872804,
  0.04110889359840405,
  0.05151352542280517,
  0.06308060418942782,
  0.07426711139814798,
  0.08273754947917003,
  0.08612711724897477,
  0.08207101352404327,
  0.06820443712085637,
  0.04216258685589523],
 [0.0032827665082694837,
  0.0032588621585557336,
  0.0032794726736168366,
  0.0033372703912255297,
  0.0034249276491545577,
  0.0035351167851766594,
  0.003660510137064576,
  0.0037940404817543355,
  0.00393568384305854,
  0.004093099579755727,
  0.00427432375704102,
  0.004484836974361799,
  0.004724418249567874,
  0.004992075139151249,
  0.005285666909254788,
  0.0055974398455355895,
  0.00591792784148287,
  0.006239136337761021,
  0.006570182375696347,
  0.006931222738977677,
  0.007342035660690445,
  0.007800530386141145,
  0.008276207572010206,
  0.008736694743800644,
  0.00916515480614243,
  0.009586649399749814,
  0.0100332328364991,
  0.010533599854643715,
  0.011096315275654496,
  0.011722480070818761,
  0.012416667696401215,
  0.013235593319384134,
  0.014276110898911107,
  0.015633858659073928,
  0.01726958017346939,
  0.018934932307426988,
  0.020363656914142197,
  0.021579787504250585,
  0.023558892779307825,
  0.027471204272740437,
  0.03435480299820879,
  0.044271095034226646,
  0.05684373786258209,
  0.07162454673898676,
  0.08678627979105874,
  0.0992375253746805,
  0.10584012645637395,
  0.10345592600266124,
  0.08894676698006437,
  0.05917449235510558],
 [0.003490027690743288,
  0.0033566316491419755,
  0.0033137759974710213,
  0.0033435919815519057,
  0.003428210847206117,
  0.003549763840255138,
  0.003690382206520453,
  0.0038327160925159783,
  0.003973448655901432,
  0.004124571381178301,
  0.004298831502323181,
  0.004504986521744182,
  0.0047428923237744615,
  0.00501120034548121,
  0.0053072549492998835,
  0.005622011360626732,
  0.005944475625050029,
  0.006265260326152383,
  0.006593659589143194,
  0.006951020146733033,
  0.007358298875069455,
  0.007813487891136386,
  0.008284747277505805,
  0.008738272332106514,
  0.00915791563204894,
  0.009575151230195828,
  0.010029400944696147,
  0.01055441841406583,
  0.011149994638364325,
  0.011803327816199643,
  0.012505670475243617,
  0.013309248749966137,
  0.01431322637809008,
  0.015616211683121347,
  0.017211044246283648,
  0.018926622098263218,
  0.020577801222919367,
  0.02222680777733027,
  0.02474671252595087,
  0.029176466916352513,
  0.03648141188460871,
  0.047082860680253844,
  0.061158291733485325,
  0.07878373087574123,
  0.09805285953439519,
  0.11524216112396383,
  0.12656092426396442,
  0.12821843757391468,
  0.11642398967333202,
  0.08738686918173413],
 [0.003623477821624202,
  0.003423683289006127,
  0.003346804348571194,
  0.003366936091627464,
  0.0034581736094830046,
  0.0035946119934458782,
  0.003750346334824149,
  0.0039001879303898626,
  0.004038316945381202,
  0.004180042650255067,
  0.004341722176291871,
  0.004535212484976221,
  0.004762330064616245,
  0.00502353286169491,
  0.005317453530328574,
  0.005633802477361925,
  0.005959568135745983,
  0.006283360522131185,
  0.00661264829655407,
  0.006967066985751894,
  0.007365904026280615,
  0.007807031733182261,
  0.008260503421204211,
  0.00869454542651121,
  0.009097037450752316,
  0.009508864092777057,
  0.009979756180480047,
  0.010550469922963079,
  0.011207987345537095,
  0.011919351868903248,
  0.012655805036153775,
  0.01345189372298596,
  0.014390897375493525,
  0.015556843099298442,
  0.017003403315156925,
  0.018737200682283163,
  0.020760841810103965,
  0.023204773640608335,
  0.026618485730804237,
  0.031637194390522344,
  0.03892979442136142,
  0.049414085728143764,
  0.0641194282196732,
  0.08394277046192869,
  0.10715372652219653,
  0.12961345559970577,
  0.14709405910802908,
  0.15536763846073975,
  0.1502062950714107,
  0.12738213035361487],
 [0.0036951298206864065,
  0.0034667698638837017,
  0.0033801718828516732,
  0.003404081538024552,
  0.003507244489836574,
  0.003658406398721973,
  0.0038263129251149782,
  0.003980564986788308,
  0.004113892851578134,
  0.00424425812553668,
  0.004390877223701372,
  0.004568335766997668,
  0.004780887463072901,
  0.005031388043993437,
  0.00532035511615011,
  0.005636877248147398,
  0.005966556270155283,
  0.006296451135226986,
  0.006630569755440221,
  0.006983854855146591,
  0.007370992201739204,
  0.007789593513731157,
  0.00821509071623513,
  0.00862146730390471,
  0.009003379311071268,
  0.009411236370164616,
  0.009904753085889114,
  0.01053125855472712,
  0.011263870298179504,
  0.012048189376376358,
  0.012833786797373768,
  0.013630310578319526,
  0.014493656136924592,
  0.015482074993971901,
  0.016723806229900367,
  0.018455569642615535,
  0.020923409191008508,
  0.02434262303831681,
  0.02882772895747969,
  0.03447262730616912,
  0.04153029613909189,
  0.05142939711560239,
  0.06612553676648246,
  0.08741692016880691,
  0.11394179735020174,
  0.14144171480645348,
  0.16555110720589863,
  0.1819044092168742,
  0.1861360555077171,
  0.17388048074676404],
 [0.003722414728111275,
  0.003494851651903245,
  0.0034154785985859555,
  0.003450376766531976,
  0.003565627354113881,
  0.003727311559704243,
  0.0039015105816756325,
  0.00405524974017201,
  0.004181086931288938,
  0.004299433019648731,
  0.004432084473364651,
  0.004595813824379649,
  0.00479618453950199,
  0.0050372434255639845,
  0.005320599885902241,
  0.005635949004214901,
  0.005969351077354468,
  0.006307922817041536,
  0.006651072083937359,
  0.00700613651221329,
  0.007380330646883424,
  0.007770946109553648,
  0.008162384293351008,
  0.008538217577008515,
  0.00890182377260906,
  0.00930999571469363,
  0.009828440914523078,
  0.010507978488582391,
  0.011310219165177365,
  0.01216369663351454,
  0.013000408101077733,
  0.013804986466320009,
  0.014602583686831646,
  0.015422226304044006,
  0.016462878435236156,
  0.018186907440251084,
  0.02107929402963829,
  0.025440908622285548,
  0.030969134016832052,
  0.03723789293363738,
  0.044088255984745264,
  0.053335683669176945,
  0.06768056387715689,
  0.08965488021243624,
  0.1183798319799129,
  0.14984998194370447,
  0.17994427877419228,
  0.20454167114175878,
  0.2195211077167859,
  0.2207615371696555],
 [0.0037227635840801836,
  0.003516888931193305,
  0.0034543244940475355,
  0.0035011702229385375,
  0.003623526068161974,
  0.003787491980013508,
  0.003959167908788802,
  0.004105644669001663,
  0.00422080974131032,
  0.004327782545216808,
  0.004451131754094667,
  0.004609104113693295,
  0.0048058413142610555,
  0.005043576539593743,
  0.005322828018722713,
  0.00563573107679618,
  0.005971863606420057,
  0.0063211662191765845,
  0.006677803400181194,
  0.007038664714248158,
  0.007400686607151479,
  0.007760862397420308,
  0.008116259283304287,
  0.008463975858544123,
  0.008817253394968938,
  0.009232869778699209,
  0.009774868919981673,
  0.010491823903753314,
  0.011339609615416482,
  0.012239729935213361,
  0.01311646128943981,
  0.01393640853734056,
  0.014698761049663044,
  0.015407615966417005,
  0.01631124537588648,
  0.018036392536180584,
  0.021242286989998596,
  0.02630018304434439,
  0.032637392719716135,
  0.03948811854327164,
  0.046409012905266674,
  0.05533983421541462,
  0.06928845605494027,
  0.09110535080887704,
  0.12043059037283214,
  0.153961300210956,
  0.18828578402952942,
  0.21999234553483415,
  0.2456692884331516,
  0.2619049164307629],
 [0.0037125073554611057,
  0.0035406983768844066,
  0.0034974924307520517,
  0.0035515752344549155,
  0.0036716325053838825,
  0.0038263499609298294,
  0.003984413318483636,
  0.0041155131166459805,
  0.004216514009300553,
  0.0043139243407739935,
  0.004435709536530725,
  0.004600755717715875,
  0.004807691878384839,
  0.005052406090534351,
  0.005330956133189216,
  0.005640229333613108,
  0.005977366095293766,
  0.006338844504599055,
  0.006713442492411672,
  0.0070849701473658395,
  0.007437482579975166,
  0.0077677788741050795,
  0.008089212985756464,
  0.008416259921957686,
  0.008772339767764965,
  0.009204997379215662,
  0.009765803905407402,
  0.010493078023818696,
  0.011345732159586714,
  0.012253236173454952,
  0.013147044479954002,
  0.013989028360302245,
  0.014764474059925738,
  0.015463772404967662,
  0.016347450213089355,
  0.018094745088805167,
  0.021420838675958776,
  0.02673772458151421,
  0.03346358870503912,
  0.04081330610221103,
  0.0482960116450679,
  0.057595604160935246,
  0.0713707181188318,
  0.09216203108980592,
  0.12008808961713949,
  0.15304709362749633,
  0.18885514096803319,
  0.22532832948590778,
  0.26028275702827747,
  0.29153452144229897],
 [0.003697640187421298,
  0.003563350816404973,
  0.0035380870574376676,
  0.0035944958402461674,
  0.0037052240945572657,
  0.0038429187500977516,
  0.003980226736594414,
  0.004090802256693323,
  0.004175539931697167,
  0.004265050402355201,
  0.004391386435504506,
  0.004573575167814034,
  0.004801581834681381,
  0.0050614394057930025,
  0.005342101922382412,
  0.005646801972571112,
  0.00598312820307044,
  0.006356785000685765,
  0.006751559855648967,
  0.0071371003249187095,
  0.007483505159155782,
  0.007787574762624489,
  0.008080795932051566,
  0.008396972093910805,
  0.008768979135654203,
  0.009227191382823715,
  0.009801565663884702,
  0.01051346429186702,
  0.011332752901970281,
  0.012210202987454767,
  0.013097714940629617,
  0.01396454910125546,
  0.014793329531017155,
  0.015571210062780259,
  0.01653650512010956,
  0.018316828042153628,
  0.021573221899759108,
  0.02672797358658537,
  0.033420760855379844,
  0.04113115712110831,
  0.049534898104052656,
  0.05975748156222041,
  0.07357419288997798,
  0.09270180335688011,
  0.11764005390345232,
  0.14777304426732735,
  0.18244362095925143,
  0.2209946304899716,
  0.26276891937023467,
  0.30710933411078695],
 [0.003678492566428556,
  0.0035760293086233024,
  0.003565006045315755,
  0.0036216255861727864,
  0.00372209074086127,
  0.003842604319048079,
  0.0039593691304000865,
  0.0040496140281569195,
  0.004118315912732771,
  0.004200721465628438,
  0.004333526698375477,
  0.00453598916465012,
  0.004788458915460803,
  0.005066021561708692,
  0.005349657873062015,
  0.005649164859911487,
  0.005983130709643019,
  0.006367069605045193,
  0.00678073545052803,
  0.007180811003160548,
  0.007524617587430393,
  0.007809249017106156,
  0.008083464976276331,
  0.008399458825420932,
  0.008797684702489981,
  0.009286936199360041,
  0.009870722934972152,
  0.010550016152780057,
  0.011310577640199958,
  0.012132529762366125,
  0.012996197954599317,
  0.013885084487471298,
  0.014785137165620802,
  0.015685779726034275,
  0.016781217388836412,
  0.01858306667301609,
  0.021630216507783687,
  0.026331481248292654,
  0.03266930910555547,
  0.04053892330739477,
  0.04990156599876919,
  0.0612064003840942,
  0.07512127733079747,
  0.09231838026637516,
  0.11353513278551942,
  0.13956876358134257,
  0.1712187127687715,
  0.2092844204627338,
  0.2545653267781571,
  0.3078608718299679],
 [0.0036553369672720415,
  0.0035698566052183456,
  0.003567103974401473,
  0.0036246456192639983,
  0.0037200480842484976,
  0.0038308779137975497,
  0.003934701652353728,
  0.00401017486878538,
  0.004065404416431502,
  0.004140624956690428,
  0.0042775949071306605,
  0.004496481975112187,
  0.004769282141916587,
  0.005061473438573811,
  0.005346978315473818,
  0.005640996554229507,
  0.0059713207075985115,
  0.006361741851627335,
  0.0067894981206775935,
  0.007201793493185171,
  0.007546612193090421,
  0.00782173011851381,
  0.008089604313456062,
  0.008416978931449971,
  0.008848853078005118,
  0.009371579718049767,
  0.00996172409462998,
  0.01059971901276294,
  0.011289170962394263,
  0.012042278864048085,
  0.012870445867366131,
  0.013772957307356285,
  0.014739770201325593,
  0.015763079556198867,
  0.0169837571594911,
  0.018773123808632018,
  0.021522320741682883,
  0.02560968077076175,
  0.03137155249069056,
  0.0391356954638405,
  0.049171809156179025,
  0.061320492633689265,
  0.07523002089301643,
  0.09060257402675735,
  0.10822362414212915,
  0.1298716877902291,
  0.1573620014708656,
  0.19250980168384776,
  0.23713032492898461,
  0.2930388077060844],
 [0.003628409182029347,
  0.0035360165730925755,
  0.0035333304309310637,
  0.0035953184801518945,
  0.003696948445362151,
  0.0038131880511689162,
  0.003919005022179271,
  0.003990590873750785,
  0.004037232990050036,
  0.004104322366334414,
  0.00423895997801201,
  0.0044634881224088225,
  0.004745009216035608,
  0.005043151357491542,
  0.005327467698409774,
  0.005616026236030103,
  0.005941695739099245,
  0.006332907047212826,
  0.0067664580259386604,
  0.0071858349446976,
  0.007535373522717517,
  0.007814014978811592,
  0.008091635007138507,
  0.00844280210337542,
  0.0089128787728264,
  0.009468459305824947,
  0.010062992006052327,
  0.010659502910797978,
  0.01127840598872052,
  0.01196139373271231,
  0.012748288910430013,
  0.013650415816569993,
  0.014657160564911169,
  0.01575901896185509,
  0.017046927942994516,
  0.018767508194168593,
  0.021180791502260335,
  0.024624355038756804,
  0.02968995883039805,
  0.037021363034753375,
  0.047124161132228136,
  0.0594828085048126,
  0.0731240834163911,
  0.08714839802542498,
  0.10215410751028832,
  0.12011271849245364,
  0.1430465128138525,
  0.17297777231641684,
  0.21192877884207895,
  0.26192181423276995],
 [0.003594363683233887,
  0.0034716597357856596,
  0.0034619104233181994,
  0.003533353136770891,
  0.0036542252670831135,
  0.0037927642051942528,
  0.003917207342043687,
  0.003997219137049827,
  0.004041057306807371,
  0.004099080168317104,
  0.004223651016002234,
  0.00444058567274534,
  0.004716469046152008,
  0.005009871712796689,
  0.005289423525760517,
  0.005572925284110071,
  0.005893178719661682,
  0.006278701357159035,
  0.00670816419784004,
  0.0071280791955195336,
  0.007485789360414568,
  0.0077817814031881545,
  0.008085577581063324,
  0.008471259893000748,
  0.008979951362635815,
  0.009563885042044428,
  0.010160458730528553,
  0.010720892404408093,
  0.011279225857146517,
  0.011900207141663205,
  0.012645635370397458,
  0.013532431666292186,
  0.014542969987108599,
  0.015659894291842075,
  0.01693536899709709,
  0.018529315228772117,
  0.02061095330246901,
  0.02347142612826159,
  0.02780152171877821,
  0.03437378654198223,
  0.043804634208899136,
  0.055556558934747396,
  0.06857486484613827,
  0.08186239556328867,
  0.09560740135528506,
  0.1110847845393328,
  0.1296096287233337,
  0.15249701751519001,
  0.18106203452280414,
  0.21661976335407773],
 [0.003543318627665268,
  0.0033848264442581837,
  0.0033679976610750326,
  0.0034529616912554995,
  0.0035998479479392763,
  0.0037687858442660505,
  0.003919904793375509,
  0.004014973488394376,
  0.004060092870728531,
  0.004109725031022387,
  0.004220650907583185,
  0.004422489105588528,
  0.004684255477525854,
  0.004966765926258562,
  0.005240073683844493,
  0.005519385151614502,
  0.005833681931724393,
  0.006208267950090643,
  0.00662565501225423,
  0.007040747104159042,
  0.007409179423408351,
  0.007732900587311391,
  0.008074021983470667,
  0.008498621868255857,
  0.009039886393517752,
  0.009642292089331568,
  0.010235510332396594,
  0.010765546432261469,
  0.011276275471815336,
  0.011847861075703263,
  0.012556634627622165,
  0.013420695876385329,
  0.014413317742962438,
  0.015507461519271269,
  0.016726576854965397,
  0.01817237035418886,
  0.019953312476980622,
  0.022309124024576122,
  0.025909745941531708,
  0.03151313259768303,
  0.03974741978462475,
  0.050281302864139225,
  0.06235345315758515,
  0.07522151260207403,
  0.08855814109383471,
  0.10241644191172049,
  0.11686358604483015,
  0.13196674448226275,
  0.14779308821311735,
  0.16440978822649271],
 [0.003464696559202257,
  0.0032847159751908618,
  0.0032685474531654727,
  0.00336989970961686,
  0.0035424814610357977,
  0.003740001423913055,
  0.0039161683147394015,
  0.004026485705674683,
  0.004074996764174161,
  0.00411869551633844,
  0.004217128433256162,
  0.00440296961285371,
  0.004648937339351134,
  0.004919637482428935,
  0.005187596455114647,
  0.005464057233086609,
  0.005772074331154376,
  0.006131921390531912,
  0.00653151084137549,
  0.0069378769131590015,
  0.00731861216094798,
  0.007676541381375336,
  0.008060257299026421,
  0.008521363846136705,
  0.009082459606286584,
  0.009687916076456356,
  0.010269049078355847,
  0.010774074006430235,
  0.011252465233498979,
  0.011791242336988384,
  0.012473120419140123,
  0.013315486104991987,
  0.014285436028466861,
  0.015349378784709378,
  0.01651005863488617,
  0.017826540122497927,
  0.01936276178589194,
  0.02130230969399609,
  0.024220957679100402,
  0.02877471242391348,
  0.03553866263709828,
  0.04448986262538632,
  0.055337325906482444,
  0.06776339894103257,
  0.08094837729108965,
  0.09361233079132793,
  0.10445831144281933,
  0.11218937124663629,
  0.11550856220385097,
  0.11311893631553549],
 [0.0033479200217236154,
  0.003180527605264401,
  0.003180515108553434,
  0.003299922757866112,
  0.003490790779477838,
  0.0037051593996640098,
  0.003895068844700028,
  0.004014387566780997,
  0.004066426069504911,
  0.0041084301861534405,
  0.004200252373522466,
  0.0043757983864562075,
  0.0046110834608218305,
  0.004874289865859584,
  0.005140170122023919,
  0.005415592923069598,
  0.005717224873818619,
  0.0060599762430071,
  0.006438312057398074,
  0.00683350686506228,
  0.007227156022282573,
  0.007621872635574029,
  0.008047572612396478,
  0.008535961643639239,
  0.009097446741756688,
  0.009684992632188767,
  0.01024197723510569,
  0.010727084138986528,
  0.01119070554296945,
  0.011717237727674424,
  0.012386926481987316,
  0.013217080010254725,
  0.014176557039616013,
  0.015233304228723099,
  0.016375321455146124,
  0.01762169108577839,
  0.01899419398929972,
  0.020615844102817124,
  0.022941483111926057,
  0.02649383724273127,
  0.03176450754401294,
  0.039015060550886785,
  0.048403960648580574,
  0.06002570437941574,
  0.07272016051220224,
  0.084177091359866,
  0.09204373158177843,
  0.09396731583731142,
  0.08759507878583649,
  0.07057425508672541],
 [0.003185109214078279,
  0.003080273371306663,
  0.003118032767877013,
  0.0032560088317795935,
  0.0034518229910046765,
  0.0036630966735425293,
  0.0038474513073834173,
  0.003964194667740146,
  0.004018239651797052,
  0.004064248858035354,
  0.004159285902236184,
  0.004335777377024374,
  0.0045711814224165875,
  0.004835518586533241,
  0.005104409865921194,
  0.005380851918477096,
  0.0056761930912233105,
  0.00600101789927314,
  0.0063570075854074,
  0.0067400993457407896,
  0.007146262037979038,
  0.0075764170277487705,
  0.008037919163634895,
  0.008538536806346064,
  0.00907600275422715,
  0.009620984011122581,
  0.0101396304469876,
  0.010609580492359452,
  0.011077309815600494,
  0.011614817828450799,
  0.012290880890961628,
  0.013125423005083606,
  0.014100761530764134,
  0.015198181162757564,
  0.016395049089898812,
  0.017662660366847608,
  0.018971809289289775,
  0.020381575661151403,
  0.022240426755084725,
  0.02495603205161372,
  0.028936613143528044,
  0.034594472078330885,
  0.04234373977540533,
  0.052518876840534454,
  0.06387959544699216,
  0.07374204946585305,
  0.07936901410307137,
  0.07802326456460161,
  0.06696757605639804,
  0.0434647237844152],
 [0.0029871976106547174,
  0.002983685574326706,
  0.003075543974012362,
  0.0032317653275811924,
  0.0034213421529027074,
  0.0036132669678464154,
  0.0037765322902818246,
  0.003881534170294153,
  0.003936625406752659,
  0.00399156504485537,
  0.004098098360337222,
  0.004284896981410259,
  0.0045291521812000895,
  0.004801089603391435,
  0.005076029899687531,
  0.005354198730010343,
  0.005643419703557246,
  0.005951572609318679,
  0.0062871687412963395,
  0.006659126816079838,
  0.0070761014627739646,
  0.007536216933626522,
  0.008023918148890986,
  0.008522740213357631,
  0.00901890114581082,
  0.009505854752746165,
  0.009978262454272267,
  0.01043721474326457,
  0.010922323875867758,
  0.011487485360012992,
  0.01218474616966939,
  0.01303814344510191,
  0.014050152301125034,
  0.015222176229870831,
  0.01652460037286834,
  0.017879731428211823,
  0.019205759925083995,
  0.02050112529538413,
  0.02202731157808352,
  0.024099615859331895,
  0.027047176436778158,
  0.0313014268118072,
  0.03733964949889474,
  0.04556366008133127,
  0.054879496208884285,
  0.06282202880423784,
  0.06687542695874867,
  0.06452385976377382,
  0.053251496310670034,
  0.03054250569079437],
 [0.002773130240039494,
  0.0028869706246869124,
  0.003039107963353613,
  0.00321255075321174,
  0.0033903074914334416,
  0.003555386675190868,
  0.0036907968016561604,
  0.0037805976617895547,
  0.0038372799537765775,
  0.00390434907504949,
  0.004026779063537472,
  0.004228208766032059,
  0.00448467540004126,
  0.004765775370654176,
  0.005046102304667234,
  0.0053246768459178345,
  0.005607971763325884,
  0.005903031288370691,
  0.006223521733974716,
  0.006587381723668931,
  0.007012042106346372,
  0.007492425890602703,
  0.007994217604390027,
  0.008481170620852688,
  0.008931011436719847,
  0.009359151899851717,
  0.009787293338238785,
  0.010238689896295141,
  0.010745899894766448,
  0.011344931493708162,
  0.012071237847552964,
  0.012951882976840506,
  0.014007472930227416,
  0.015257574584714585,
  0.016669371805750927,
  0.018128854790879768,
  0.019515046711907343,
  0.020778072527048285,
  0.022101118841901517,
  0.023715051465854974,
  0.025867183873332547,
  0.02892638806316448,
  0.03331601892461253,
  0.03940001313801945,
  0.04636290623901882,
  0.05230808690897795,
  0.05529896610492009,
  0.05339895478386867,
  0.04467146390284699,
  0.027179904418878513],
 [0.002561881734714597,
  0.0027863219040544394,
  0.002994752991023833,
  0.0031836931357315788,
  0.00334966047828648,
  0.00348917315879734,
  0.0035987493173729557,
  0.00367560837641116,
  0.0037359350484351436,
  0.0038166028957717364,
  0.00395544031128933,
  0.00417077560878904,
  0.004437429850190754,
  0.004724337281092163,
  0.00500568200883173,
  0.005281310092471331,
  0.005558896466525921,
  0.005846765875823305,
  0.006160774868954723,
  0.0065216392227459,
  0.006949434028924729,
  0.007436179371081226,
  0.00793745088493495,
  0.008406422897257865,
  0.008817218282568132,
  0.00920045790401965,
  0.009596191831772747,
  0.010042757182660155,
  0.010568227387759309,
  0.01119687026813763,
  0.011953082365854583,
  0.012863279600792038,
  0.013955432413878296,
  0.015256565745955637,
  0.01673457527192583,
  0.018265706304482796,
  0.019718333646789914,
  0.021015634599968393,
  0.02226042133931078,
  0.023592255320265126,
  0.025166804496565335,
  0.02725877391083913,
  0.03019622138128604,
  0.034267596535142365,
  0.03897357190445854,
  0.04309267155454498,
  0.04537671656191379,
  0.044577528003077196,
  0.039446926954547316,
  0.028736734492836404],
 [0.002372254546523662,
  0.002678012022718726,
  0.0029286938945852986,
  0.0031307063718982123,
  0.0032904556644323037,
  0.0034143479819624075,
  0.0035087895342633553,
  0.003580610889774634,
  0.0036481188588403488,
  0.003742139451010392,
  0.0038940488836727853,
  0.004117577095095085,
  0.004387082844186442,
  0.0046715977112535955,
  0.004945948018027939,
  0.0052132854749904044,
  0.005485403221889285,
  0.005774256937959971,
  0.006093667274460374,
  0.006458658090404882,
  0.006883647956908963,
  0.007358745299659035,
  0.007842478805584786,
  0.008291300687142717,
  0.00868243951274134,
  0.009049157440166855,
  0.00943406832850162,
  0.009877819678150246,
  0.010409276947808608,
  0.011052937004135188,
  0.011832980708968403,
  0.01276896769292246,
  0.013876900153664211,
  0.015171979358220006,
  0.016626841306937163,
  0.018148145836810434,
  0.019636883634007937,
  0.021019489702008072,
  0.022305801934827733,
  0.023522720593712403,
  0.024717593835002636,
  0.026088892539726027,
  0.027902821241412558,
  0.030401552120867578,
  0.03334597536449346,
  0.03605579410660427,
  0.03783439764377506,
  0.03798517527258098,
  0.03581151628959711,
  0.0306168099913986],
 [0.0022146600115133743,
  0.0025621747483585682,
  0.0028362384901612515,
  0.0030481626019918918,
  0.003209258448920955,
  0.003330837396018909,
  0.0034242108083562205,
  0.00350094296379053,
  0.0035794378457591346,
  0.003685560733598481,
  0.003845479765103575,
  0.004069533603146441,
  0.00433274326136057,
  0.004605351043968745,
  0.004864126195816644,
  0.005117742427702994,
  0.005384606774416647,
  0.005682278904622918,
  0.006018440204312235,
  0.006394398973878352,
  0.00681106176964939,
  0.007257846575778912,
  0.00770924731481748,
  0.008138782209917775,
  0.008533209659658567,
  0.00891499666112491,
  0.00931256978227077,
  0.009755313321840686,
  0.010278350116643478,
  0.010918930761334484,
  0.011712393237709588,
  0.012665404990154585,
  0.013762562826583644,
  0.01498784120467706,
  0.016321937572926132,
  0.01774047001557913,
  0.019218615698162734,
  0.020715248590686953,
  0.0221358030831365,
  0.02337478102187489,
  0.024358676904502125,
  0.025250430916662375,
  0.02631895890122045,
  0.02781680687437056,
  0.02966311340275293,
  0.03147138532041934,
  0.03284382807243083,
  0.03338264710384848,
  0.03269004785973335,
  0.030368235785146507],
 [0.002087510627495591,
  0.002444465093268499,
  0.0027256970576075034,
  0.0029435867608140005,
  0.003110514443189391,
  0.0032388603450350744,
  0.003341004706652447,
  0.0034294920913329014,
  0.003521310969234182,
  0.003638297578256659,
  0.003802467058733366,
  0.004021761066063799,
  0.004272721451998565,
  0.004527641469063046,
  0.004766089082707568,
  0.0050031919247316295,
  0.00526492606909539,
  0.005575176142003252,
  0.0059334828700611,
  0.00632368123819835,
  0.0067294935203208225,
  0.0071404363610870075,
  0.00755355349770866,
  0.007966394772454496,
  0.008378375052958865,
  0.008793939158487658,
  0.009218371379875764,
  0.009662411972091587,
  0.010169492274242516,
  0.010795164954946935,
  0.011591006288795027,
  0.012548796645207179,
  0.013614285387396282,
  0.014732785846278545,
  0.01589449405477293,
  0.017159174233902612,
  0.0185925602125731,
  0.020200019061250143,
  0.02178904411360789,
  0.02312664815953778,
  0.02402579940495433,
  0.024639105302317457,
  0.025271400619663435,
  0.02621339251490195,
  0.027462383323723007,
  0.02874671371258956,
  0.0297847789132356,
  0.030294974157395217,
  0.029995694676802444,
  0.02860533570319132],
 [0.0019882961116735286,
  0.0023309626856406356,
  0.0026063798420400183,
  0.002825499928574168,
  0.002999275292945585,
  0.003138658282856764,
  0.0032546012460101994,
  0.0033581882670068544,
  0.003464066087795078,
  0.0035907678808642087,
  0.0037569649748273844,
  0.00396892902130397,
  0.004205266354847035,
  0.004440840011294545,
  0.004658374199074451,
  0.004879019477908241,
  0.0051356494097908,
  0.005457875188526995,
  0.005837349673728949,
  0.0062412364771425,
  0.006636872019860823,
  0.00701417767820163,
  0.007392413484144072,
  0.007792784591265404,
  0.008226959812964356,
  0.008680888564542973,
  0.009136227833650732,
  0.009584423591817924,
  0.010075575512808469,
  0.010681531122148816,
  0.011468369766146631,
  0.012415328385601254,
  0.013434792465586513,
  0.014438878519038861,
  0.015426743826303824,
  0.016532458857211007,
  0.017901676069080268,
  0.01958409803090234,
  0.021314917083867808,
  0.022764983806318927,
  0.023662137730111516,
  0.024155402371662027,
  0.024582577155900346,
  0.0252671234805119,
  0.02623153206835931,
  0.027222396421269997,
  0.02797610814943538,
  0.028229058863046927,
  0.027717640172296068,
  0.02617824368737425],
 [0.0019145061812504039,
  0.0022277471536670916,
  0.0024875970885747563,
  0.0027024231854820197,
  0.002880592643897506,
  0.00303047266332984,
  0.003160430443287642,
  0.0032789614854174917,
  0.003398031059971405,
  0.0035333895373004087,
  0.003700927723650848,
  0.0039057070063237623,
  0.004128626908652597,
  0.004347317695421271,
  0.004547519065291033,
  0.004754610599064743,
  0.005006065100368162,
  0.005335302582620156,
  0.005728595017337754,
  0.0061417962844884195,
  0.0065311260792069396,
  0.006886733549741076,
  0.007242843404009443,
  0.007636597882863014,
  0.008087988059997149,
  0.008570748511578732,
  0.009050893855929774,
  0.009506656143934677,
  0.009989471924544054,
  0.01057792080011639,
  0.011344033573686291,
  0.012261185938857824,
  0.013226808690638713,
  0.01413818445897238,
  0.015000919961345085,
  0.01598852425093445,
  0.017288922159525485,
  0.018977782416848225,
  0.020762814051542098,
  0.02227644976183619,
  0.02321086827372593,
  0.023699808799666794,
  0.024074919269090004,
  0.02465381420925057,
  0.025458306577617405,
  0.02623905058461573,
  0.02673667376427615,
  0.02669180365062933,
  0.0258450677777059,
  0.02393709367953649],
 [0.0018627617375624216,
  0.0021393271923285536,
  0.002377020835701346,
  0.00258163415709007,
  0.0027589586459039984,
  0.0029147857915524026,
  0.003054907083444554,
  0.003185244790630211,
  0.0033152579516784767,
  0.0034582637951936297,
  0.0036277305021070595,
  0.0038277423003508556,
  0.004041446579325664,
  0.004249157836960633,
  0.00443906905840138,
  0.004637884216050926,
  0.004884056105085515,
  0.005211822793547507,
  0.005606392769210248,
  0.006021342532576773,
  0.006410643721007166,
  0.006764123808931776,
  0.007118188271348095,
  0.007512326072797206,
  0.007968090557150696,
  0.008458662908785584,
  0.008949151261759089,
  0.00941603783470849,
  0.009903956165985116,
  0.010482809209887152,
  0.011216752955769257,
  0.01208333205156691,
  0.012993417563083464,
  0.01385778474526554,
  0.014684364765717092,
  0.015631680456660885,
  0.01687118408158958,
  0.01847329471428975,
  0.020177258854372434,
  0.021654572494959515,
  0.02262686057901228,
  0.02318623231939855,
  0.023590849075328976,
  0.024088094976559327,
  0.024696876336863047,
  0.025226655827257236,
  0.025479151490744145,
  0.025256081370326044,
  0.02435916350900518,
  0.022590115949783832],
 [0.001825112841669441,
  0.0020619468169803907,
  0.0022737045168482583,
  0.0024638686539997625,
  0.002635921941161626,
  0.0027933470910605713,
  0.002939626816423318,
  0.003078378769921009,
  0.0032168698838879154,
  0.003366348015724197,
  0.0035382243178022515,
  0.003735826082069471,
  0.003944444440185359,
  0.004146930113115072,
  0.004333349889392171,
  0.004529043558386567,
  0.00477011189652716,
  0.005088843244716509,
  0.005473175809877426,
  0.005882434809268382,
  0.00627622888071806,
  0.006643723469609494,
  0.007012479359756411,
  0.007412602231363378,
  0.007861306615138444,
  0.008341039762139563,
  0.008828446453552987,
  0.009308020970679112,
  0.009811290284623967,
  0.010387220333292516,
  0.011081102452770617,
  0.011882817367640608,
  0.012739590604025448,
  0.013598538808032844,
  0.014466080214761203,
  0.015440550522244446,
  0.016628175832375445,
  0.01806776788427231,
  0.019577163625289613,
  0.020928993499494026,
  0.021926501848507175,
  0.022599188850567286,
  0.02307796321956315,
  0.02348880617839453,
  0.02385085983760383,
  0.024085327815172277,
  0.024109792243209086,
  0.023841835253823523,
  0.023199038979124797,
  0.022098985551222146],
 [0.0017921117160765955,
  0.001989141754121472,
  0.0021738772972201567,
  0.0023477187709836744,
  0.0025120666010230532,
  0.0026683212129493196,
  0.002817883032373496,
  0.0029622952678354255,
  0.0031069625111043377,
  0.003261501658487858,
  0.003435709959306707,
  0.0036324351569768038,
  0.0038390197293811925,
  0.004040708190356182,
  0.004228976814149734,
  0.004425763464806249,
  0.00466229914179449,
  0.004966802352300636,
  0.005332444979772788,
  0.005729788182666752,
  0.00612947717478497,
  0.006520074687098403,
  0.006913418939118447,
  0.007322896569237335,
  0.007757549396811608,
  0.008214701314997082,
  0.008689720558801376,
  0.00918085115792457,
  0.009703568348863198,
  0.010279736423140645,
  0.010930286611972536,
  0.011662032111865991,
  0.012470918028253002,
  0.013352713391017384,
  0.014309396599776612,
  0.015352570577235288,
  0.016494661049853407,
  0.017727174584308517,
  0.01897304768614401,
  0.020141188959035435,
  0.02114633827442043,
  0.021946332217040537,
  0.02251832344609905,
  0.022841706714374955,
  0.022938388037910432,
  0.022869243141022177,
  0.022696588736592874,
  0.022482741537505197,
  0.022290018256641835,
  0.022180735606885455],
 [0.0017543097144731496,
  0.0019144461593174516,
  0.00207376670381508,
  0.0022317753593597287,
  0.002387976137345094,
  0.0025418730491648727,
  0.0026929701062127567,
  0.0028409276319684624,
  0.0029896332120397454,
  0.0031475858664321192,
  0.0033234896361773463,
  0.0035200473083118232,
  0.0037265720795898387,
  0.003930565447446319,
  0.004124564096417062,
  0.0043257173074605654,
  0.0045586831026493,
  0.004846137970404198,
  0.005187701738796121,
  0.005568118971153949,
  0.005971984678864949,
  0.006387717973533284,
  0.006808705608202421,
  0.007228675142303916,
  0.007646729671664179,
  0.008076470050991437,
  0.008533916732096875,
  0.009032775622766233,
  0.009572884329669724,
  0.010148938315923553,
  0.010757509185997627,
  0.0114233672860492,
  0.012192990409425512,
  0.013112570253805427,
  0.01417762932129452,
  0.015305152860859735,
  0.016405377298699647,
  0.017417469397489,
  0.01837542549056613,
  0.019332641921848682,
  0.02032292774211135,
  0.02123532966374394,
  0.02189401149060014,
  0.022132594299956756,
  0.021977658318397944,
  0.02163266775995477,
  0.021307627503567107,
  0.021212542428174847,
  0.021557417412717876,
  0.02255225733613606],
 [0.0017022581905483693,
  0.0018313941881339887,
  0.0019696002636310694,
  0.0021146292704458483,
  0.0022642340619845654,
  0.0024161674916534596,
  0.0025681824128587663,
  0.0027182092099151214,
  0.0028689793654061413,
  0.0030284617825044797,
  0.003204865557971089,
  0.0034011403193134992,
  0.003608501123487969,
  0.003816575263147836,
  0.0040187259999371495,
  0.004226578458500115,
  0.004457329040853384,
  0.004727287953131512,
  0.005042447546847221,
  0.005402143493112044,
  0.005805347468615051,
  0.006241193841048914,
  0.00668603796577655,
  0.007115404006447961,
  0.007518758209190151,
  0.007923168453755932,
  0.008361978128032103,
  0.00886204159152548,
  0.009411332198010476,
  0.00998340684813327,
  0.0105559739274685,
  0.011169213891996368,
  0.01191139832120236,
  0.012870371155983237,
  0.014034093779846117,
  0.01523570961234411,
  0.016295062143590344,
  0.017104606906904376,
  0.01779481149218649,
  0.01854483543619876,
  0.01949282813693924,
  0.02047384843560316,
  0.02118710908872989,
  0.0213472666505961,
  0.02098686805968148,
  0.020429867627117893,
  0.020010995076803456,
  0.020064980132636294,
  0.02092655251851452,
  0.022930441958336246],
 [0.0016265084979915196,
  0.0017335199961367386,
  0.0018576055036661626,
  0.0019948713555599552,
  0.002141423886798282,
  0.002293369432361307,
  0.0024468143272291937,
  0.0025980733492704023,
  0.002749098349915529,
  0.002907990549652442,
  0.003083139934244857,
  0.003278191973220799,
  0.003486206493752257,
  0.003698811016223086,
  0.003910076788452996,
  0.0041260202900754986,
  0.004356302218168531,
  0.0046106901545868925,
  0.00490018386382589,
  0.0052365780669231055,
  0.005631161619692333,
  0.006075042801780076,
  0.00653311461060905,
  0.006968549217554309,
  0.007363545778883516,
  0.00775161900692387,
  0.008174847901199683,
  0.008666896290523687,
  0.009211005924852366,
  0.009771722856261825,
  0.010318884589007765,
  0.010901962931513642,
  0.011631732337242934,
  0.012618377857137084,
  0.013842105375962593,
  0.015081653070914735,
  0.01609845314920167,
  0.01675454169564526,
  0.01724172014463559,
  0.01781925255035061,
  0.0186925973442634,
  0.019669555777543816,
  0.020379697976151762,
  0.020471521481749164,
  0.019984214642376135,
  0.019315108697659376,
  0.01887477798897355,
  0.019073796857693323,
  0.020322739645193367,
  0.023032180692848327],
 [0.0015176119904918672,
  0.0016143577388913592,
  0.0017340099509184,
  0.0018710924660199722,
  0.002020129123643058,
  0.002175643763234642,
  0.0023321602242417057,
  0.0024844533976293077,
  0.0026340875442799096,
  0.0027900333108235083,
  0.002961614974555571,
  0.0031536800532726917,
  0.0033610878230593767,
  0.0035773460854344272,
  0.003797230725707598,
  0.004021716174337312,
  0.00425366789635654,
  0.004496782428874652,
  0.004764412149631921,
  0.0050761390109692,
  0.005451023207753852,
  0.00588380536786155,
  0.006337634141468141,
  0.0067735768315078035,
  0.00717100315023827,
  0.00755864419412856,
  0.007973469206192237,
  0.008445586946082228,
  0.008963999481162327,
  0.00950246717680124,
  0.010039444923238031,
  0.010624005406407162,
  0.01135958303120662,
  0.012348852116853241,
  0.013564979510175152,
  0.014780395475797948,
  0.01575028788020981,
  0.016333228346802293,
  0.016726665901543945,
  0.017197376312569203,
  0.017958793249443133,
  0.01883011893449156,
  0.01945385988852922,
  0.0194911565088721,
  0.01898789544709702,
  0.018342656926727062,
  0.01796706277274903,
  0.018272734810149734,
  0.019671294863915993,
  0.022574364759034585],
 [0.0013661200217386766,
  0.0014674415719635064,
  0.00159504113238582,
  0.0017418834531438198,
  0.0019009332843757094,
  0.0020651553762196916,
  0.0022275144788139684,
  0.002381282702586839,
  0.0025280443272112864,
  0.0026784512089651795,
  0.002843592888460149,
  0.003030082342708146,
  0.003234544744085998,
  0.00345225384954421,
  0.0036788020754439485,
  0.003911339483436152,
  0.004147491337179197,
  0.004386002630099102,
  0.004638633864165107,
  0.004925542643632398,
  0.005266528308456657,
  0.005662022051428114,
  0.006087295157122034,
  0.006515952904193282,
  0.006931041092748396,
  0.007341066499003299,
  0.007758785197602377,
  0.008196360784522475,
  0.008662406837907274,
  0.009164220646243535,
  0.009710858682781905,
  0.010337732318483065,
  0.0111005409767528,
  0.012054055694717963,
  0.013166031583014978,
  0.014269349066220056,
  0.015185303901290935,
  0.01580662144346608,
  0.016260163216542056,
  0.016720689771119495,
  0.01732797373783774,
  0.017963205151372025,
  0.018391676561525728,
  0.01839196944742107,
  0.018016107854459222,
  0.01756677826946877,
  0.017355935960801528,
  0.017695536196809324,
  0.018897534245843985,
  0.02127388537625731],
 [0.0011625839454212136,
  0.0012863056509188375,
  0.0014369265750664628,
  0.0016058351682494223,
  0.001784419880853051,
  0.0019640691632626826,
  0.0021361714658636494,
  0.002292494611737996,
  0.0024350660774216603,
  0.002577105387024957,
  0.0027323758855155133,
  0.0029098766247661295,
  0.003107976889508795,
  0.00332360768731479,
  0.003553405101405046,
  0.003792563589522618,
  0.004035837802398295,
  0.004278788612364558,
  0.004526350467325247,
  0.004789505283294765,
  0.005079272997457809,
  0.0054042333646145534,
  0.005769796256338948,
  0.006181143491495585,
  0.0066335703759078934,
  0.007095708405181392,
  0.007531739030022725,
  0.007917465032165803,
  0.008298321966054133,
  0.00874556410108074,
  0.009326329620261992,
  0.01004553466954749,
  0.010860196747540856,
  0.01172625035031752,
  0.012608576995013268,
  0.013485926081407392,
  0.014338238777121218,
  0.015140675568727243,
  0.01585272654326042,
  0.016430675974266448,
  0.016836696694806513,
  0.01707648167311086,
  0.01717522973080475,
  0.017159758012852237,
  0.017087049245077847,
  0.017041738681032332,
  0.01710948408580268,
  0.017375943224475884,
  0.017926773862138937,
  0.018847633763878817]])

In [25]:
x_start_home = np.array([17.84313725, 57.25490196, 54.50980392, 51.17647059, 66.07843137,
       81.76470588, 72.54901961, 73.7254902 , 88.23529412, 85.29411765,
       86.07843137])
y_start_home = np.array([52.93048163, 81.79306526, 40.51561202, 60.4426571 , 76.4554639 ,
       25.21448812, 51.42804328, 63.17076627, 77.87882426, 67.91529901,
       48.69993411])
x_start_away = np.array([98.43137255, 88.43137255, 86.47058824, 85.88235294, 86.47058824,
       87.45098039, 81.76470588, 78.82352941, 58.03921569, 68.82352941,
       57.05882353])
y_start_away = np.array([49.0557742 , 65.06857828, 53.32585529, 36.60137102, 44.42985302,
       70.76202245, 65.06857828, 56.17257601, 41.58313229, 72.18538281,
       56.5284161 ])
vx_start_home = np.array([18.62745098, 58.82352941, 55.68627451, 52.35294118, 66.8627451 ,
       85.29411765, 74.11764706, 75.88235294, 91.37254902, 89.01960784,
       90.19607843])
vy_start_home = np.array([52.81187008, 80.96277081, 41.2272922 , 60.32404555, 76.81130399,
       28.41704894, 49.64884283, 62.103246  , 76.09962381, 66.49193593,
       42.29481248])
vx_start_away = np.array([98.43137255, 90.58823529, 90.        , 88.43137255, 88.43137255,
       90.98039216, 83.92156863, 81.37254902, 58.23529412, 69.60784314,
       56.07843137])
vy_start_away = np.array([45.85321338, 61.51017737, 49.41161429, 37.6688913 , 43.00649266,
       70.05034226, 64.71273819, 54.39337556, 41.93897238, 72.18538281,
       57.47732482])

In [7]:
style = {'description_width': 'auto'}

# Create ToggleButtons
class_buttons = widgets.ToggleButtons(
    options=['Spearman', 'Fernandez', 'Modified Fernandez'],
    description='Pitch control model:',
    style = style)
display(class_buttons)

class_buttons_transition = widgets.ToggleButtons(
    options=['No', 'Yes'],
    description='Use transition model (for red team):',
    style=style)
display(class_buttons_transition)

class_buttons_value = widgets.ToggleButtons(
    options=['No', 'Yes'],
    description='Use scoring model (for red team):',
    style=style)
display(class_buttons_value)


ToggleButtons(description='Pitch control model:', options=('Spearman', 'Fernandez', 'Modified Fernandez'), sty…

ToggleButtons(description='Use transition model (for red team):', options=('No', 'Yes'), style=ToggleButtonsSt…

ToggleButtons(description='Use scoring model (for red team):', options=('No', 'Yes'), style=ToggleButtonsStyle…

In [33]:
fig = plt.figure()
fig.min_aspect_ratio = 105/74.8
fig.max_aspect_ratio = 115.5/74.8
plt.plot([[0,0],
         [0,100],
         [100,100],
         [100,0]],
         
         [[0,100],
         [0,0],
         [0,100],
         [100,100]],         
         colors=['black'],stroke_width = 1,
         axes_options={'x': {'visible': False}, 'y': {'visible': False}})
plt.scatter([25,35,50,70],
            [105,105,105,105],
           colors = ['white'],
           stroke = 'orange')
plt.plot([[25,26],[35,37],[50,57],[70,82]],
         [[105,105],[105,105],[105,105],[105,105]],
         colors = ['black'],
         stroke_width = 1.)
plt.scatter([26,37,57,82],
            [105,105,105,105],
            colors = ['black'],
            default_size = 4,
            stroke_width = 1.,
            marker = 'triangle-up',
            scales={'rotation': LinearScale(min=0, max=180)},
            rotation = [90.] * 4)

plt.label(['Velocity guide:','Walk','Jog','Sprint','Usain Bolt'],
            x=[1,22.5,33,49,69],
            y=[108,110,110,110,110],
           colors = ['black'],
         default_size = 12)
fig

Figure(axes=[Axis(scale=LinearScale(), visible=False), Axis(orientation='vertical', scale=LinearScale(), visib…

In [35]:
xx = np.linspace(0,100,50)
yy = np.linspace(0,100,50)
XX,YY = np.meshgrid(xx,yy)
xxyy = np.dstack((XX,YY))
targets = xxyy * np.array([105/100,68/100])

cm=plt2.get_cmap('coolwarm')

home_pos = np.c_[x_start_home,y_start_home] * np.array([105/100,68/100])
away_pos = np.c_[x_start_away,y_start_away] * np.array([105/100,68/100])
home_v = np.c_[vx_start_home,vy_start_home] * np.array([105/100,68/100])
away_v = np.c_[vx_start_away,vy_start_away] * np.array([105/100,68/100])
ball_pos = np.array([50.,50.]) * np.array([105/100,68/100])

hmap = plt.plot([],[],
                 fill='inside',
                 axes_options={'x': {'visible': False}, 'y': {'visible': False}},
                 stroke_width=0,
                 close_path=False,
                 display_legend=False)

plt.ylim(0,110)
plt.xlim(0,100)

scat_home = plt.scatter(x_start_home, y_start_home, colors = ['white'],stroke='orange', enable_move=True)
scat_away = plt.scatter(x_start_away, y_start_away, colors=['white'],stroke='blue', enable_move=True)
scat_ball = plt.scatter([67.84313725], [77.24621935], colors=['white'], enable_move=True,stroke='black')

scat_v_home = plt.scatter(vx_start_home, vy_start_home, colors=['black'], enable_move=True,
                          marker = 'triangle-up',
                         default_size = 4,
                          scales={'rotation': LinearScale(min=0, max=180)},
                         rotation = [90.]*len(scat_home.x))
scat_v_away = plt.scatter(vx_start_away, vy_start_away, colors=['black'], enable_move=True,
                          marker='triangle-up',
                         default_size = 4,
                          scales={'rotation': LinearScale(min=0, max=180)},
                         rotation = [-90.]*len(scat_home.x))

lines_v_home = plt.plot(np.vstack([x_start_home,vx_start_home]).T,
                        np.vstack([y_start_home,vy_start_home]).T,
                        colors = ['black'],
                       stroke_width = 1.)

lines_v_away = plt.plot(np.vstack([x_start_away,vx_start_away]).T,
                        np.vstack([y_start_away,vy_start_away]).T,
                        colors = ['black'],
                       stroke_width = 1.)

plt.plot([[50,50],
         [0,16.5/105*100],
         [0,16.5/105*100],
         [100,100 - 16.5/105*100],
         [100,100 - 16.5/105*100],
         [16.5/105*100,16.5/105*100],
         [100-16.5/105*100,100 - 16.5/105*100],
         [0,5.5/105*100],
         [0,5.5/105*100],
         [100,100 - 5.5/105*100],
         [100,100 - 5.5/105*100],
         [5.5/105*100,5.5/105*100],
         [100-5.5/105*100,100 - 5.5/105*100]],
         
         [[0,100],
         [50-20/68*100,50-20/68*100],
         [50+20/68*100,50+20/68*100],
         [50-20/68*100,50-20/68*100],
         [50+20/68*100,50+20/68*100],
         [50-20/68*100,50+20/68*100],
         [50-20/68*100,50+20/68*100],
         [50-9/68*100,50-9/68*100],
         [50+9/68*100,50+9/68*100],
         [50-9/68*100,50-9/68*100],
         [50+9/68*100,50+9/68*100],
         [50-9/68*100,50+9/68*100],
         [50-9/68*100,50+9/68*100]],
         
         colors=['black'],stroke_width = 1)

plt.plot(50 + 9.15/105*100 * np.sin(np.linspace(0,2*np.pi,500)),
         50 + 9.15/68*100 * np.cos(np.linspace(0,2*np.pi,500)),
         colors=['black'],stroke_width = 1)

v_home_x = vx_start_home - x_start_home
v_home_y = vy_start_home - y_start_home

v_away_x = vx_start_away - x_start_away
v_away_y = vy_start_away - y_start_away

def update_hmap(change=None):
    global X_new
    with hmap.hold_sync():
        new_home_pos_x = scat_home.x
        new_home_pos_y = scat_home.y
        new_away_pos_x = scat_away.x
        new_away_pos_y = scat_away.y
        
        scat_v_home.y, scat_v_home.x = new_home_pos_y + v_home_y, new_home_pos_x + v_home_x
        scat_v_away.y, scat_v_away.x = new_away_pos_y + v_away_y, new_away_pos_x + v_away_x
        
        lines_v_home.x, lines_v_home.y = np.vstack([new_home_pos_x,new_home_pos_x + v_home_x]).T, np.vstack([new_home_pos_y,new_home_pos_y + v_home_y]).T
        lines_v_away.x, lines_v_away.y = np.vstack([new_away_pos_x,new_away_pos_x + v_away_x]).T, np.vstack([new_away_pos_y,new_away_pos_y + v_away_y]).T

        
        home_pos = np.c_[scat_home.x,scat_home.y] * np.array([105/100,68/100])
        away_pos = np.c_[scat_away.x,scat_away.y] * np.array([105/100,68/100])
        home_v = np.c_[v_home_x,v_home_y] * np.array([105/100,68/100])
        away_v = np.c_[v_away_x,v_away_y] * np.array([105/100,68/100])
        ball_pos = np.c_[scat_ball.x,scat_ball.y] * np.array([105/100,68/100])
        if class_buttons.value == 'Spearman':
            X_new = get_pitch_control(home_pos,
                                      away_pos,
                                      home_v,
                                      away_v,
                                      ball_pos,
                                      targets,
                                      xx,
                                      yy,
                                      xT_values,
                                      class_buttons_transition.value,
                                      class_buttons_value.value)
        elif class_buttons.value == 'Fernandez':
            X_new = get_pitch_control_fernandez(home_pos,
                                                away_pos,
                                                home_v,
                                                away_v,
                                                ball_pos,
                                                targets,
                                                xx,
                                                yy,
                                                xT_values,
                                                class_buttons_transition.value,
                                                class_buttons_value.value)            
        else:
            X_new = get_pitch_control_fernandez_modified(home_pos,
                                                         away_pos,
                                                         home_v,
                                                         away_v,
                                                         ball_pos,
                                                         targets,
                                                         xx,
                                                         yy,
                                                         xT_values,
                                                         class_buttons_transition.value,
                                                         class_buttons_value.value)
        
        contours_x,contours_y,levels = get_contours(X_new,xx,yy)
        l = np.concatenate(levels)
        l /= np.max(l)
        cols= (cm(l)[:,:3]*255) * 0.8 + 25.5
        hmap.fill_colors=['rgb(' + str(int(c[0])) + ',' + str(int(c[1])) + ',' + str(int(c[2])) + ')' for c in cols]
        hmap.x = contours_x
        hmap.y = contours_y
                
update_hmap()
        
def update_v_markers_angle_home(change=None):
    #with scat_v_home.hold_sync():
        global v_home_x, v_home_y
        v_home_y = scat_v_home.y - scat_home.y
        v_home_x = scat_v_home.x - scat_home.x
        lines_v_home.x, lines_v_home.y = np.vstack([scat_home.x,scat_home.x + v_home_x]).T, np.vstack([scat_home.y,scat_home.y + v_home_y]).T
        scat_v_home.rotation = [np.degrees(np.arctan2(-yyy,xxx)) + 90. for yyy,xxx in zip(v_home_y,v_home_x)]
        
def update_v_markers_angle_away(change=None):
    #with scat_v_home.hold_sync():
        global v_away_x, v_away_y
        v_away_y = scat_v_away.y - scat_away.y
        v_away_x = scat_v_away.x - scat_away.x
        lines_v_away.x, lines_v_away.y = np.vstack([scat_away.x,scat_away.x + v_away_x]).T, np.vstack([scat_away.y,scat_away.y + v_away_y]).T
        scat_v_away.rotation = [np.degrees(np.arctan2(-yyy,xxx)) + 90. for yyy,xxx in zip(v_away_y,v_away_x)]
    
        

# update line on change of x or y of scatter
scat_v_home.observe(update_v_markers_angle_home, ['x','y'])
scat_v_away.observe(update_v_markers_angle_away, ['x','y'])
scat_home.observe(update_hmap, names=['x','y'])
scat_v_home.observe(update_hmap, ['x','y'])
scat_away.observe(update_hmap, names=['x','y'])
scat_v_away.observe(update_hmap, ['x','y'])
scat_ball.observe(update_hmap, names=['x','y'])

In [123]:
from IPython.display import display, clear_output
button = widgets.Button(description="Get red team control")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        print(get_pitch_control_proportion(X_new))

button.on_click(on_button_clicked)

Button(description='Get red team control', style=ButtonStyle())

Output()